# this script will download the entire database then save them to a sqlite database (except for price column)
* method: scrape data from index to index
    * for example we scrape from 020000000000 to 030000000000
* method: decode json and save to sqlite database

In [1]:
from __future__ import absolute_import, division, print_function, \
    with_statement
import requests
from peewee import * 
import json

In [22]:
def parse_json_save_to_sqlite(json_string):
    try:
        json_obj = json.loads(json_string)
    except Exception as e:
        logging.error(e)
        logging.error("there is some problem with json loading")
    
    total = len(json_obj['features'])
    for index, each_building in enumerate(json_obj['features']):
        building_info = each_building['properties']
        
        building = PropertyModel()
        if len(str(building_info['wobj_obj_id'])) > 0:
            try:
                building.identificatie = building_info['wobj_obj_id']
            except Exception as e:
                print("this building has no identification code. ")
                building.identificatie = "none"
        else:
            building.identificatie = "none"
            print("this building has no identification code. ")
            
        if len(str(building_info['wobj_huisnummer'])) > 0:
            try:
                building.house_number = building_info['wobj_huisnummer']
            except Exception as e:
                building.house_number = "none"
                print("this building has no house number. ")
        else:
            building.house_number = "none"
            print("this building has no house number. ")
        
        if len(str(building_info['wobj_huisletter'])) > 0:
            try:
                building.house_number_ext = building_info['wobj_huisletter']
            except Exception as e:
                building.house_number_ext = "none"
                print("this building has no house number extension")
        else:
            building.house_number_ext = "none"
            print("this building has no house number extension")
            
        if len(str(building_info['wobj_postcode'])) > 0:
            try:
                building.postcode = building_info['wobj_postcode']
            except Exception as e:
                building.postcode = "none"
                print('this building has no postcode. ')
        else:
            building.postcode = "none"
            print('this building has no postcode. ')
            
        if len(str(building_info['wobj_woonplaats'])) > 0:
            try:
                building.plaatsnaam = building_info['wobj_woonplaats']
            except Exception as e:
                building.plaatsnaam = "none"
                print("this building has no plaatsnaam. ")
        else:
            building.plaatsnaam = "none"
            print("this building has no plaatsnaam. ")
        
        if len(str(building_info['wobj_straat'])) > 0:
            try:
                building.street = building_info['wobj_straat']
            except Exception as e:
                building.street = "none"
                print("this building has no street name. ")
        else:
            building.street = "none"
            print("this building has no street name. ")
            
        if len(str(building_info['wobj_bag_bouwjaar'])) > 0:
            try:
                building.bouwjaar = building_info['wobj_bag_bouwjaar']
            except Exception as e:
                building.bouwjaar = "none"
                print("this building has no bouwjaar. ")
        else:
            building.bouwjaar = "none"
            print("this building has no bouwjaar. ")
            
        if len(str(building_info['wobj_bag_gebruiksdoel'])) > 0:
            try:
                building.gebruiksdoel = building_info['wobj_bag_gebruiksdoel']
            except Exception as e:
                building.gebruiksdoel = "none"
                print("this building has no gebruiksdoel. ")
        else:
            building.gebruiksdoel = "none"
            print("this building has no gebruiksdoel. ")
            
        if len(str(building_info['wobj_oppervlakte'])) > 0:
            try:
                building.oppervlakte = building_info['wobj_oppervlakte']
            except Exception as e:
                building.oppervlakte = "none"
                print("this building has no oppervlakte. ")
        else:
            building.oppervlakte = "none"
            print("this building has no oppervlakte. ")
            
        # init other empty fields
        building.price_2015 = "none"
        building.price_2016 = "none"
        building.pdf_link = "none"
        
        try:
            building.save()
        except:
            pass
        
        print("Saving property to database now, process: {:2.2f}% ...".format((index+1)*100/(total)))

In [19]:
class BaseModel(Model):
    class Meta:
        database = SqliteDatabase("netherland_properties.db")
class PropertyModel(BaseModel):
    identificatie = CharField(unique=True)
    house_number = CharField(null=True)
    house_number_ext = CharField(null=True)
    postcode = CharField(null=True)
    plaatsnaam = CharField(null=True)
    street = CharField(null=True)
    
    price_2015 = CharField(null=True)
    price_2016 = CharField(null=True)
    
    bouwjaar = CharField(null=True)
    gebruiksdoel = CharField(null=True)
    oppervlakte = CharField(null=True)
    
    pdf_link = CharField(null=True)

db = SqliteDatabase("netherland_properties.db")
db.connect()
try:
    db.drop_tables([PropertyModel])
except:
    pass
db.create_tables([PropertyModel])
db.close()

In [8]:
def scrape_obj_from_id_to_id(f=None, t=None):
    if f is None:
        raise Exception("From id is None, check the script please. ")
    if t is None:
        raise Exception("To id is None, check the script please. ")
        
    # format from_id and to_id
    from_id = '%012d'%f
    to_id = '%012d'%t
    
    # initiate a request object
    s = requests.Session()
    s.get("https://www.wozwaardeloket.nl/index.jsp?a=1&accept=true&")
    
    xml_obj = \
    """
    <wfs:GetFeature
        xmlns:wfs="http://www.opengis.net/wfs" service="WFS" version="1.1.0" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <wfs:Query typeName="wozloket:woz_woz_object" srsName="EPSG:28992"
            xmlns:WozViewer="http://WozViewer.geonovum.nl"
            xmlns:ogc="http://www.opengis.net/ogc">
            <ogc:Filter
                xmlns:ogc="http://www.opengis.net/ogc">
                <ogc:And>
                    <ogc:PropertyIsGreaterThan matchCase="true">
                        <ogc:PropertyName>wobj_obj_id</ogc:PropertyName>
                        <ogc:Literal>%s</ogc:Literal>
                    </ogc:PropertyIsGreaterThan>
                    <ogc:PropertyIsLessThan matchCase="true">
                        <ogc:PropertyName>wobj_obj_id</ogc:PropertyName>
                        <ogc:Literal>%s</ogc:Literal>
                    </ogc:PropertyIsLessThan>
                </ogc:And>
            </ogc:Filter>
        </wfs:Query>
    </wfs:GetFeature>
    """
    xml_obj = xml_obj%(str(from_id), str(to_id))
    
    response = None
    try:
        response = s.post(url="https://www.wozwaardeloket.nl/woz-proxy/wozloket", data=xml_obj)
        print("scraping woz obj from id %s to id %s . "%(str(from_id), str(to_id)))
    except Exception as e:
        logging.error(e)
        logging.error("request has met problem. ")
        logging.error("from_id=%s"%str(from_id))
        logging.error("to_id=%s"%str(to_id))
        
    return response.text

In [7]:
json_string = scrape_obj_from_id_to_id(20000000000, 30000000000)

In [23]:
parse_json_save_to_sqlite(json_string=json_string)

Saving property to database now, process: 0.01% ...
Saving property to database now, process: 0.01% ...
Saving property to database now, process: 0.02% ...
Saving property to database now, process: 0.03% ...
Saving property to database now, process: 0.03% ...
Saving property to database now, process: 0.04% ...
Saving property to database now, process: 0.05% ...
Saving property to database now, process: 0.05% ...
Saving property to database now, process: 0.06% ...
Saving property to database now, process: 0.07% ...
Saving property to database now, process: 0.07% ...
Saving property to database now, process: 0.08% ...
Saving property to database now, process: 0.09% ...
Saving property to database now, process: 0.09% ...
Saving property to database now, process: 0.10% ...
Saving property to database now, process: 0.11% ...
Saving property to database now, process: 0.11% ...
Saving property to database now, process: 0.12% ...
Saving property to database now, process: 0.13% ...
Saving prope

Saving property to database now, process: 1.07% ...
Saving property to database now, process: 1.08% ...
Saving property to database now, process: 1.09% ...
Saving property to database now, process: 1.09% ...
Saving property to database now, process: 1.10% ...
Saving property to database now, process: 1.11% ...
Saving property to database now, process: 1.11% ...
Saving property to database now, process: 1.12% ...
Saving property to database now, process: 1.13% ...
Saving property to database now, process: 1.13% ...
Saving property to database now, process: 1.14% ...
Saving property to database now, process: 1.15% ...
Saving property to database now, process: 1.15% ...
Saving property to database now, process: 1.16% ...
Saving property to database now, process: 1.17% ...
Saving property to database now, process: 1.17% ...
Saving property to database now, process: 1.18% ...
Saving property to database now, process: 1.19% ...
Saving property to database now, process: 1.19% ...
Saving prope

Saving property to database now, process: 2.98% ...
Saving property to database now, process: 2.99% ...
Saving property to database now, process: 2.99% ...
Saving property to database now, process: 3.00% ...
Saving property to database now, process: 3.01% ...
Saving property to database now, process: 3.01% ...
Saving property to database now, process: 3.02% ...
Saving property to database now, process: 3.03% ...
Saving property to database now, process: 3.03% ...
Saving property to database now, process: 3.04% ...
Saving property to database now, process: 3.05% ...
Saving property to database now, process: 3.05% ...
Saving property to database now, process: 3.06% ...
Saving property to database now, process: 3.07% ...
Saving property to database now, process: 3.07% ...
Saving property to database now, process: 3.08% ...
Saving property to database now, process: 3.09% ...
Saving property to database now, process: 3.09% ...
Saving property to database now, process: 3.10% ...
Saving prope

Saving property to database now, process: 4.29% ...
Saving property to database now, process: 4.29% ...
Saving property to database now, process: 4.30% ...
Saving property to database now, process: 4.31% ...
Saving property to database now, process: 4.31% ...
Saving property to database now, process: 4.32% ...
Saving property to database now, process: 4.33% ...
Saving property to database now, process: 4.33% ...
Saving property to database now, process: 4.34% ...
Saving property to database now, process: 4.35% ...
Saving property to database now, process: 4.35% ...
Saving property to database now, process: 4.36% ...
Saving property to database now, process: 4.37% ...
Saving property to database now, process: 4.37% ...
Saving property to database now, process: 4.38% ...
Saving property to database now, process: 4.39% ...
Saving property to database now, process: 4.39% ...
Saving property to database now, process: 4.40% ...
Saving property to database now, process: 4.41% ...
Saving prope

Saving property to database now, process: 6.13% ...
Saving property to database now, process: 6.13% ...
Saving property to database now, process: 6.14% ...
Saving property to database now, process: 6.15% ...
Saving property to database now, process: 6.15% ...
Saving property to database now, process: 6.16% ...
Saving property to database now, process: 6.17% ...
Saving property to database now, process: 6.17% ...
Saving property to database now, process: 6.18% ...
Saving property to database now, process: 6.19% ...
Saving property to database now, process: 6.19% ...
Saving property to database now, process: 6.20% ...
Saving property to database now, process: 6.21% ...
Saving property to database now, process: 6.21% ...
Saving property to database now, process: 6.22% ...
Saving property to database now, process: 6.23% ...
Saving property to database now, process: 6.23% ...
Saving property to database now, process: 6.24% ...
Saving property to database now, process: 6.25% ...
Saving prope

...

Saving property to database now, process: 93.17% ...
Saving property to database now, process: 93.17% ...
Saving property to database now, process: 93.18% ...
Saving property to database now, process: 93.19% ...
Saving property to database now, process: 93.19% ...
Saving property to database now, process: 93.20% ...
Saving property to database now, process: 93.21% ...
Saving property to database now, process: 93.21% ...
Saving property to database now, process: 93.22% ...
Saving property to database now, process: 93.23% ...
Saving property to database now, process: 93.23% ...
Saving property to database now, process: 93.24% ...
Saving property to database now, process: 93.25% ...
Saving property to database now, process: 93.25% ...
Saving property to database now, process: 93.26% ...
Saving property to database now, process: 93.27% ...
Saving property to database now, process: 93.27% ...
Saving property to database now, process: 93.28% ...
Saving property to database now, process: 93.2

Saving property to database now, process: 94.87% ...
Saving property to database now, process: 94.87% ...
Saving property to database now, process: 94.88% ...
Saving property to database now, process: 94.89% ...
Saving property to database now, process: 94.89% ...
Saving property to database now, process: 94.90% ...
Saving property to database now, process: 94.91% ...
Saving property to database now, process: 94.91% ...
Saving property to database now, process: 94.92% ...
Saving property to database now, process: 94.93% ...
Saving property to database now, process: 94.93% ...
Saving property to database now, process: 94.94% ...
Saving property to database now, process: 94.95% ...
Saving property to database now, process: 94.95% ...
Saving property to database now, process: 94.96% ...
Saving property to database now, process: 94.97% ...
Saving property to database now, process: 94.97% ...
Saving property to database now, process: 94.98% ...
Saving property to database now, process: 94.9

Saving property to database now, process: 96.51% ...
Saving property to database now, process: 96.51% ...
Saving property to database now, process: 96.52% ...
Saving property to database now, process: 96.53% ...
Saving property to database now, process: 96.53% ...
Saving property to database now, process: 96.54% ...
Saving property to database now, process: 96.55% ...
Saving property to database now, process: 96.55% ...
Saving property to database now, process: 96.56% ...
Saving property to database now, process: 96.57% ...
Saving property to database now, process: 96.57% ...
Saving property to database now, process: 96.58% ...
Saving property to database now, process: 96.59% ...
Saving property to database now, process: 96.59% ...
Saving property to database now, process: 96.60% ...
Saving property to database now, process: 96.61% ...
Saving property to database now, process: 96.61% ...
Saving property to database now, process: 96.62% ...
Saving property to database now, process: 96.6

Saving property to database now, process: 98.35% ...
Saving property to database now, process: 98.36% ...
Saving property to database now, process: 98.37% ...
Saving property to database now, process: 98.37% ...
Saving property to database now, process: 98.38% ...
Saving property to database now, process: 98.39% ...
Saving property to database now, process: 98.39% ...
Saving property to database now, process: 98.40% ...
Saving property to database now, process: 98.41% ...
Saving property to database now, process: 98.41% ...
Saving property to database now, process: 98.42% ...
Saving property to database now, process: 98.43% ...
Saving property to database now, process: 98.43% ...
Saving property to database now, process: 98.44% ...
Saving property to database now, process: 98.45% ...
Saving property to database now, process: 98.45% ...
Saving property to database now, process: 98.46% ...
Saving property to database now, process: 98.47% ...
Saving property to database now, process: 98.4

Saving property to database now, process: 99.99% ...
Saving property to database now, process: 100.00% ...
